In [1]:
import sys
# S-Box
sBox  = [0x9, 0x4, 0xa, 0xb, 0xd, 0x1, 0x8, 0x5, 0x6, 0x2, 0x0, 0x3, 0xc, 0xe, 0xf, 0x7] 
# Inverse S-Box
sBoxI = [0xa, 0x5, 0x9, 0xb, 0x1, 0x7, 0x8, 0xf, 0x6, 0x0, 0x2, 0x3, 0xc, 0x4, 0xd, 0xe]
w = [None] * 6

In [2]:
def mult(p1, p2):
    """Multiply two polynomials in GF(2^4)/x^4 + x + 1"""
    p = 0
    while p2:
        if p2 & 0b1:
            p ^= p1
        p1 <<= 1
        if p1 & 0b10000:
            p1 ^= 0b11
        p2 >>= 1
    return p & 0b1111
 
def intToVec(n):
    return [n >> 12, (n >> 4) & 0xf, (n >> 8) & 0xf,  n & 0xf]            
 
def vecToInt(m):
    return (m[0] << 12) + (m[2] << 8) + (m[1] << 4) + m[3]
 
def addKey(s1, s2):
    """Add two keys in GF(2^4)"""  
    return [i ^ j for i, j in zip(s1, s2)]
          
def shiftRow(s):
    """ShiftRow function"""
    return [s[0], s[1], s[3], s[2]]

def sub4NibList(sbox, s):
    """Nibble substitution function"""
    return [sbox[e] for e in s]
 

In [3]:
def sub2Nib(b):
        """Swap each nibble and substitute it using sBox"""
        return sBox[b >> 4] + (sBox[b & 0x0f] << 4)

In [4]:
get_bin = lambda x: format(x, 'b')

In [5]:
def keyExp(key):
    Rcon1, Rcon2 = 0b10000000,0b00110000
    w[0] = (key & 0xff00) >> 8
    w[1] = key & 0x00ff
    w[2] = w[0] ^ Rcon1 ^ sub2Nib(w[1])
    w[3] = w[2] ^ w[1]
    w[4] = w[2] ^ Rcon2 ^ sub2Nib(w[3])
    w[5] = w[4] ^ w[3]

In [6]:
plain_text = 0b1101011100101000
print(plain_text)
key = 0b0100101011110101
keyExp(key)

55080


In [7]:
print('value of w[0]')
print(get_bin(w[0]).zfill(8))
print('value of w[1]')
print(get_bin(w[1]).zfill(8))
print('value of w[2]')
print(get_bin(w[2]).zfill(8))
print('value of w[3]')
print(get_bin(w[3]).zfill(8))
print('value of w[4]')
print(get_bin(w[4]).zfill(8))
print('value of w[5]')
print(get_bin(w[5]).zfill(8))

value of w[0]
01001010
value of w[1]
11110101
value of w[2]
11011101
value of w[3]
00101000
value of w[4]
10000111
value of w[5]
10101111


In [8]:
def mixCol(s):
        return [s[0] ^ mult(4, s[2]), s[1] ^ mult(4, s[3]),
                s[2] ^ mult(4, s[0]), s[3] ^ mult(4, s[1])]    

In [9]:
print(get_bin(w[0]<<8))

100101000000000


In [10]:
#After Round 0
state = intToVec(((w[0] << 8) + w[1]) ^ plain_text)
print('After Round 1:-')
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))

After Round 1:-
1001 1101 1101 1101


In [11]:
#Round 1
#SubNibble operation
print('Performing the subnibble operations:-')
state = sub4NibList(sBox, state)
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
#Shift Row operation
print('Perfroming the state Row shift operations:-')
state = shiftRow(state)
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
#Mix column operation
print('Result afte mix row operation:-')
state = mixCol(state)
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
# XOR with Round 1 key
print('After round key 1 xor')    
state = addKey(intToVec((w[2] << 8) + w[3]), state)
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))

Performing the subnibble operations:-
0010 1110 1110 1110
Perfroming the state Row shift operations:-
0010 1110 1110 1110
Result afte mix row operation:-
1111 0110 0011 0011
After round key 1 xor
0010 1011 0001 1011


In [12]:
#Round 2
#SubNibble operation
print('Performing the subnibble operations:-')
state = sub4NibList(sBox, state)
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
#Shift Row operation
print('Perfroming the state Row shift operations:-')
state = shiftRow(state)
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
print('After round key 2 xor')    
state = addKey(intToVec((w[4] << 8) + w[5]), state)
print('Cipher Text is:-')
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))    

Performing the subnibble operations:-
1010 0011 0100 0011
Perfroming the state Row shift operations:-
1010 0011 0100 0011
After round key 2 xor
Cipher Text is:-
0010 0100 1110 1100


In [13]:
def decrypt_MixCol(s):
        return [mult(9, s[0]) ^ mult(2, s[2]), mult(9, s[1]) ^ mult(2, s[3]),
                mult(9, s[2]) ^ mult(2, s[0]), mult(9, s[3]) ^ mult(2, s[1])]

In [14]:
#Decryption starts 
ctext = vecToInt(state)
state = intToVec(((w[4] << 8) + w[5]) ^ ctext)
# XOR with Round 2 key
print('After round key 2')
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
# Shift Rows
state = shiftRow(state)
print('After shift row operation:-')
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
state = sub4NibList(sBoxI, state)
print('After Using inverse sbox:-')
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
state = decrypt_MixCol(addKey(intToVec((w[2] << 8) + w[3]), state))
print('After XOR with Round 1 key and mixing operation')
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
state = shiftRow(state)
state = sub4NibList(sBoxI,state)
print('After shift row and using inverse sbox ')
print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
decrypt_text = vecToInt(addKey(intToVec((w[0] << 8) + w[1]), state))
print('Decrypted plain text is:-'+str(decrypt_text))


After round key 2
1010 0011 0100 0011
After shift row operation:-
1010 0011 0100 0011
After Using inverse sbox:-
0010 1011 0001 1011
After XOR with Round 1 key and mixing operation
0010 1110 1110 1110
After shift row and using inverse sbox 
1001 1101 1101 1101
Decrypted plain text is:-55080


In [15]:
def encryptAES(plaintext):
    print('Encryption starts:-\n')
    #After Round 0
    state = intToVec(((w[0] << 8) + w[1]) ^ plain_text)
    print('After Round 0:-')
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    #Round 1
    #SubNibble operation
    print('Performing the subnibble operations:-')
    state = sub4NibList(sBox, state)
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    #Shift Row operation
    print('Perfroming the state Row shift operations:-')
    state = shiftRow(state)
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    #Mix column operation
    print('Result afte mix row operation:-')
    state = mixCol(state)
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    # XOR with Round 1 key
    print('After round key 1 xor')    
    state = addKey(intToVec((w[2] << 8) + w[3]), state)
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    #Round 2
    #SubNibble operation
    print('Performing the subnibble operations:-')
    state = sub4NibList(sBox, state)
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    #Shift Row operation
    print('Perfroming the state Row shift operations:-')
    state = shiftRow(state)
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    print('After round key 2 xor')    
    state = addKey(intToVec((w[4] << 8) + w[5]), state)
    print('Cipher Text is:-')
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    ctext = vecToInt(state)
    print('\n')
    return ctext

In [16]:
def decryptAES(ctext):
    #Decryption starts 
    print('Decryption starts :- \n')
    state = intToVec(((w[4] << 8) + w[5]) ^ ctext)
    # XOR with Round 2 key
    print('After round key 2')
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    # Shift Rows
    state = shiftRow(state)
    print('After shift row operation:-')
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    state = sub4NibList(sBoxI, state)
    print('After Using inverse sbox:-')
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    state = decrypt_MixCol(addKey(intToVec((w[2] << 8) + w[3]), state))
    print('After XOR with Round 1 key and mixing operation')
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    state = shiftRow(state)
    state = sub4NibList(sBoxI,state)
    print('After shift row and using inverse sbox ')
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    state = addKey(intToVec((w[0] << 8) + w[1]), state)
    decrypt_text = vecToInt(state)
    print('Binary decrypted text:-')
    print(str(get_bin(state[0]).zfill(4)) + " " + str(get_bin(state[2]).zfill(4)) +" "+ str(get_bin(state[1]).zfill(4)) +" "+ str(get_bin(state[3]).zfill(4)))
    print('Decrypted plain text is:-'+str(decrypt_text)) 
    print('\n')
    return decrypt_text

In [17]:
def BinaryToDecimal(binary):  
         
    binary1 = binary  
    decimal, i, n = 0, 0, 0
    while(binary != 0):  
        dec = binary % 10
        decimal = decimal + dec * pow(2, i)  
        binary = binary//10
        i += 1
    return (decimal) 

In [19]:
key = 0b0100101011110101
keyExp(key)
input_text = input('plain text ')
print(input_text)
blocks = []
str_data = ""
for char in input_text:
    char = bin(ord(char))[2:].zfill(8)
    blocks.append(char)
print("plaintext as blocks", blocks) 

for i in range(0,len(blocks),2):
  res = str(blocks[i] + blocks[i+1])
  print(res)                                                                                                                                          
  plain_text = int(res,2)
  print('plain_text is:-')
  print(plain_text)
  cipher_text = encryptAES(plain_text)  
  b = decryptAES(cipher_text)
  b=get_bin(b).zfill(16)
  b = str(b)
  for i in range(0, len(b), 8): 
    temp_data = int(b[i:i + 8]) 
    decimal_data = BinaryToDecimal(temp_data) 
    str_data = str_data + chr(decimal_data)


print(str_data)    


plain text 23
23
plaintext as blocks ['00110010', '00110011']
0011001000110011
plain_text is:-
12851
Encryption starts:-

After Round 0:-
0111 1000 1100 0110
Performing the subnibble operations:-
0101 0110 1100 1000
Perfroming the state Row shift operations:-
0101 1000 1100 0110
Result afte mix row operation:-
0011 1111 0111 0011
After round key 1 xor
1110 0010 0101 1011
Performing the subnibble operations:-
1111 1010 0001 0011
Perfroming the state Row shift operations:-
1111 0011 0001 1010
After round key 2 xor
Cipher Text is:-
0111 0100 1011 0101


Decryption starts :- 

After round key 2
1111 0011 0001 1010
After shift row operation:-
1111 1010 0001 0011
After Using inverse sbox:-
1110 0010 0101 1011
After XOR with Round 1 key and mixing operation
0101 1000 1100 0110
After shift row and using inverse sbox 
0111 1000 1100 0110
Binary decrypted text:-
0011 0010 0011 0011
Decrypted plain text is:-12851


23
